In [115]:
import pandas as pd
import sklearn
import numpy as np

In [116]:
df=pd.read_csv('./train.csv')
dftest=pd.read_csv("./test.csv")
dftest["Survived"]=-1
df=pd.concat([df,dftest])
df.info()
# dftest

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       1309 non-null int64
Ticket         1309 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 132.9+ KB


C:\Users\noufal.samsudin\AppData\Local\Continuum\anaconda3\envs\manipalenvname\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [117]:
for col in df.columns:
    print(col,df[col].nunique())

Age 98
Cabin 186
Embarked 3
Fare 281
Name 1307
Parch 8
PassengerId 1309
Pclass 3
Sex 2
SibSp 7
Survived 3
Ticket 929


In [118]:
df.columns

Index(['Age', 'Cabin', 'Embarked', 'Fare', 'Name', 'Parch', 'PassengerId',
       'Pclass', 'Sex', 'SibSp', 'Survived', 'Ticket'],
      dtype='object')

In [119]:
df['Embarked'].value_counts()

S    914
C    270
Q    123
Name: Embarked, dtype: int64

In [120]:
dropcols=['PassengerId','Name']
catcols=['Pclass','Sex','Ticket','Cabin','Embarked']
num_cols=['Age','SibSp','Parch','Fare']
target=['Survived']
df.isnull().sum()

Age             263
Cabin          1014
Embarked          2
Fare              1
Name              0
Parch             0
PassengerId       0
Pclass            0
Sex               0
SibSp             0
Survived          0
Ticket            0
dtype: int64

In [121]:
# df[col].mode()

In [122]:

# dfbackup=df.copy()
for col in num_cols:
    print(col)
    df.loc[df[col].isnull(),col]=df[col].median()
for col in catcols:
    print(col)
    df.loc[df[col].isnull(),col]=df[col].mode()[0]
df.isnull().sum()

Age
SibSp
Parch
Fare
Pclass
Sex
Ticket
Cabin
Embarked


Age            0
Cabin          0
Embarked       0
Fare           0
Name           0
Parch          0
PassengerId    0
Pclass         0
Sex            0
SibSp          0
Survived       0
Ticket         0
dtype: int64

In [123]:
from sklearn.preprocessing import LabelEncoder
dffeats=pd.DataFrame()
for col in catcols:
  print("looking at {}: val count {}".format(col,df[col].nunique()))
  if (df[col].nunique()>3):
    print("reducing...")
    freqs=df[col].value_counts()
    small_cats=freqs[freqs<3].index
    dffeats[col]=df[col].replace(small_cats, 'other')
  else:
    dffeats[col]=df[col]  
  
  enc = LabelEncoder()
  dffeats[col]=enc.fit_transform(dffeats[col].values)

looking at Pclass: val count 3
looking at Sex: val count 2
looking at Ticket: val count 929
reducing...
looking at Cabin: val count 186
reducing...
looking at Embarked: val count 3


In [124]:
from sklearn.preprocessing import MinMaxScaler

for col in num_cols:
  print(col)
  dffeats[col]=df[col]
  upperlimit=dffeats[col].quantile(.9)
  lowerlimit=dffeats[col].quantile(.1)
  dffeats.loc[dffeats[col]>upperlimit,col]=upperlimit
  dffeats.loc[dffeats[col]<lowerlimit,col]=lowerlimit
  scaler=MinMaxScaler()
  dffeats[col]=scaler.fit_transform(dffeats[col].values.reshape(-1,1)).flatten()
dffeats.head()

Age
SibSp
Parch
Fare


,Pclass,Sex,Ticket,Cabin,Embarked,Age,SibSp,Parch,Fare
0,2,1,84,7,2,0.18750,1.0,0.0,0.000000
1,0,0,84,16,0,0.68750,1.0,0.0,0.904376
2,2,0,84,7,2,0.31250,0.0,0.0,0.005039
3,0,0,84,16,2,0.59375,1.0,0.0,0.646274
4,2,1,84,7,2,0.59375,0.0,0.0,0.006813


In [125]:
for col in dropcols+target:
    dffeats[col]=df[col]
testdata=dffeats[dffeats[target[0]]==-1]
traindata=dffeats[dffeats[target[0]]!=-1]
X=traindata[num_cols+catcols].values
y=traindata[target[0]].values

In [126]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn.metrics import accuracy_score, log_loss,roc_curve,auc,roc_auc_score,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# import lightgbm as lgb
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [127]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=314,stratify=y)

In [128]:
classifiers = [
    LogisticRegression(),
    KNeighborsClassifier(3),
    KNeighborsClassifier(9),
    KNeighborsClassifier(27),
    KNeighborsClassifier(51),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]

# Logging for Visual Comparison

for clf in classifiers:
    
    clf.fit(X_train, y_train)
    
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    print('****Results****')
    print("Cross val score; {}".format(np.mean(cross_val_score(clf, X, y, cv=10, scoring='roc_auc'))))
    cat_train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, cat_train_predictions)
    print("Training Accuracy: {:.4%}".format(accuracy_score(y_train,clf.predict(X_train))))
    print("Testing Accuracy: {:.4%}".format(acc))
    

C:\Users\noufal.samsudin\AppData\Local\Continuum\anaconda3\envs\manipalenvname\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\noufal.samsudin\AppData\Local\Continuum\anaconda3\envs\manipalenvname\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentatio

LogisticRegression
****Results****


C:\Users\noufal.samsudin\AppData\Local\Continuum\anaconda3\envs\manipalenvname\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\noufal.samsudin\AppData\Local\Continuum\anaconda3\envs\manipalenvname\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentatio

Cross val score; 0.8518840788840787
Training Accuracy: 81.2199%
Testing Accuracy: 76.1194%
KNeighborsClassifier
****Results****
Cross val score; 0.802751181280593
Training Accuracy: 86.0353%
Testing Accuracy: 78.3582%
KNeighborsClassifier
****Results****
Cross val score; 0.8217342330871741
Training Accuracy: 79.9358%
Testing Accuracy: 80.9701%
KNeighborsClassifier
****Results****
Cross val score; 0.818849418555301
Training Accuracy: 78.6517%
Testing Accuracy: 78.3582%
KNeighborsClassifier
****Results****
Cross val score; 0.8176287808052514
Training Accuracy: 75.9230%
Testing Accuracy: 74.6269%
DecisionTreeClassifier
****Results****
Cross val score; 0.7686627054862349
Training Accuracy: 97.7528%
Testing Accuracy: 76.8657%
RandomForestClassifier
****Results****
Cross val score; 0.8640182214299861
Training Accuracy: 97.7528%
Testing Accuracy: 77.2388%
AdaBoostClassifier
****Results****
Cross val score; 0.8519869281045752
Training Accuracy: 83.7881%
Testing Accuracy: 78.3582%
GradientBoost

In [129]:
output=classifiers[-1].predict(testdata[num_cols+catcols].values)
testdata["Survived"]=output
testdata.head()

C:\Users\noufal.samsudin\AppData\Local\Continuum\anaconda3\envs\manipalenvname\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Pclass,Sex,Ticket,Cabin,Embarked,Age,SibSp,Parch,Fare,PassengerId,Name,Survived
0,2,1,84,7,1,0.578125,0.0,0.0,0.003679,892,"Kelly, Mr. James",0
1,2,0,84,7,2,0.968750,1.0,0.0,0.000000,893,"Wilkes, Mrs. James (Ellen Needs)",0
2,1,1,84,7,1,1.000000,0.0,0.0,0.030057,894,"Myles, Mr. Thomas Francis",0
3,2,1,84,7,2,0.343750,0.0,0.0,0.015507,895,"Wirz, Mr. Albert",0
4,2,0,84,7,2,0.187500,1.0,0.5,0.066962,896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0


In [130]:
# testdata[["PassengerId","Survived"]].to_csv("QDAoutput.csv",index=None)

In [131]:
from sklearn.ensemble import VotingClassifier

In [132]:
votingC = VotingClassifier(estimators=
[("{}_{}".format(i,clf.__class__.__name__ ),clf) for i,clf in enumerate(classifiers)], voting='soft', n_jobs=4)

votingC = votingC.fit(X_train, y_train)

In [133]:
print("Cross val score; {}".format(np.mean(cross_val_score(votingC, X, y, cv=10, scoring='roc_auc'))))

Cross val score; 0.866873864697394


In [134]:
output=votingC.predict(testdata[num_cols+catcols].values)
testdata["Survived"]=output
testdata[["PassengerId","Survived"]].to_csv("Voringoutliernoutput.csv",index=None)

C:\Users\noufal.samsudin\AppData\Local\Continuum\anaconda3\envs\manipalenvname\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
